In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

df = pd.read_csv("kddcup.data_10_percent_corrected", header=None)

df.columns = [ "duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label" ]

df['label'] = df['label'].apply(lambda x: 0 if x == 'normal.' else 1)

df = pd.get_dummies(df, columns=['protocol_type', 'service', 'flag'])

X = df.drop('label', axis=1)
Y = df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

num_components = 6 
pca = PCA(n_components=num_components)
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)



In [2]:
import pennylane as qml
from pennylane import numpy as np

num_qubits = num_components

dev = qml.device("default.qubit", wires=num_qubits)

#circuit1
"""
def circuit(x, weights):
    
    for i in range(num_qubits):
        qml.RY(x[i], wires=i)

  
    for i in range(num_qubits):
        qml.Rot(*weights[i], wires=i)

        
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    return qml.expval(qml.PauliZ(0))
"""

#circuit2
def circuit(x, weights):
    
    for i in range(num_qubits):
        qml.RY(x[i], wires=i)

    num_pqc_layers = weights.shape[0]

  
    for k in range(num_pqc_layers):
        for i in range(num_qubits):
            qml.Rot(*weights[k, i], wires=i)

        for i in range(num_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

        qml.CNOT(wires=[num_qubits - 1, 0])

    return qml.expval(qml.PauliZ(0))

In [3]:
new_num_pqc_layers =  3
#weights_shape = (num_qubits, 3)
weights_shape = (new_num_pqc_layers, num_qubits, 3)
weights = np.random.uniform(0, np.pi, weights_shape, requires_grad=True)

qnode = qml.QNode(circuit, dev, interface="autograd")

In [4]:
def cost(weights, X, Y):
    loss = 0
    for i in range(len(X)):
        pred = qnode(X[i], weights)
        loss += (pred - Y[i])**2
    return loss / len(X)


In [5]:
from pennylane.optimize import NesterovMomentumOptimizer 

X_train_small = np.array(X_train[:500])  
Y_train_small = 2 * np.array(Y_train[:500]) - 1  

#opt = NesterovMomentumOptimizer(0.5)
opt = qml.GradientDescentOptimizer(stepsize=0.5)
steps = 15

for i in range(steps):
    weights = opt.step(lambda w: cost(w, X_train_small, Y_train_small), weights)
    print(f"Step {i + 1}: Cost = {cost(weights, X_train_small, Y_train_small)}")



Step 1: Cost = 0.9085082923468486
Step 2: Cost = 0.7631168325190587
Step 3: Cost = 0.620811789250216
Step 4: Cost = 0.5143364688325257
Step 5: Cost = 0.45265059265175417
Step 6: Cost = 0.4177081625616152
Step 7: Cost = 0.39579471080601186
Step 8: Cost = 0.38054975005441866
Step 9: Cost = 0.3690807722559656
Step 10: Cost = 0.35995566919465677
Step 11: Cost = 0.3523955109752723
Step 12: Cost = 0.34594025166919895
Step 13: Cost = 0.34029878874087915
Step 14: Cost = 0.335275841269999
Step 15: Cost = 0.3307334882556492


In [6]:
X_test_np = np.array(X_test[:50])
Y_test_np = 2 * np.array(Y_test[:50]) - 1

predictions = [np.sign(qnode(x, weights)) for x in X_test_np]
acc = np.mean(predictions == Y_test_np)
print("Test accuracy:", acc)


Test accuracy: 0.98


In [7]:
#circuit1:
#0.82 acc on 200 rows, 0.5 stepsize and 30 steps
#0.82 acc on 500 rows, 0.5 stepsize and 15 steps
#0.76 acc on 500 rows, 0.2 stepsize and 15 steps
#0.82 acc on 500 rows, 0.8 stepsize and 15 steps

#circuit2:
#0.98 acc on 500 rows, 0.5 stepsize and 15 steps